In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
skincare = pd.read_csv(f"{DATA_PATH}아모레크롤링_스킨케어_완료.csv")
skincare

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,0103*******,5,2022.09.27,40대,여성,지성,모공,남편이 극지성이에요. 원래 수퍼화산송이 미셀라만 꾸준히 사용해 오다가 지성 전용이라...
1,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,lsm2***,5,2022.08.15,40대,여성,건성,건조함,여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 ...
2,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,gpdl***,5,2022.12.24,20대,여성,복합성,트러블,이니스프리 폼클렌징은 피부에 자극적이지 않아 좋아요 선크림만 사용해도 꼭 폼까지 써...
3,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,zoll***,5,2022.09.30,30대,여성,건성,건조함,저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁...
4,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,snow****,5,2022.09.30,50대 이상,여성,지성,모공,여름이라서 모공이 넓어지는것 같아서 구매해서 사용해보니 촉촉하고 피부땡기지 않아서 ...
...,...,...,...,...,...,...,...,...,...,...,...
112742,스킨케어,프리메라,모이스처 클렌징 티슈 300g,drea****,5,2020.01.11,40대,여성,복합성,모공,자극적이지 않고 순하게 화장을 지워주어 좋아요.
112743,스킨케어,프리메라,모이스처 클렌징 티슈 300g,kim4***,5,2022.05.11,50대 이상,여성,복합성,칙칙함,순하고 화장도 잘지워지고 기름지지않아서 좋아요 ~~^^
112744,스킨케어,프리메라,모이스처 클렌징 티슈 300g,hyej*****,4,2022.04.12,40대,여성,수분부족지성,탄력없음,클렌징티슈는 이것만 사용해요 자극 덜하고 향도 좋아요~
112745,스킨케어,프리메라,모이스처 클렌징 티슈 300g,sooh*****,4,2019.08.31,40대,여성,수분부족지성,모공,제 인생템 중 하나입니다 없으면 불안해져 쟁여놓고 쓴답니다~


In [6]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120895 files and direc

In [6]:
import matplotlib.pyplot as plt # 맷플롯립 import 하기
plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기

In [7]:
from datetime import datetime, timedelta
current_date = datetime.now()

In [8]:
from collections import Counter

In [9]:
def chatbot_with_recommendation(df):
    model_name = "noahkim/KoT5_news_summarization"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        # 여러 키워드에 대한 처리
        keywords = title.split()
        condition = lambda x: all(keyword.lower() in x.lower() for keyword in keywords)
        # 중복 제거를 위해 unique() 사용
        matching_titles = [t for t in df['상품명'].unique().tolist() if condition(t)]
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")

        # 사용자가 상품을 선택하도록 함
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]

        # 선택한 상품의 리뷰 가져오기
        reviews = df[df['상품명'] == selected_title]['리뷰'].to_list()
        reviews_text = ' '.join(reviews)
        # 리뷰 요약
        inputs = tokenizer(reviews_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=40, max_length=200, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(f"{selected_title}의 전체 리뷰 요약: {summary}")
        # 비슷한 상품을 보여줄지 물어봄
        show_similar = input("비슷한 상품을 보여드릴까요? (Y/N): ")

        if show_similar.lower() == 'y':
        # 선택한 상품의 나이, 피부타입, 피부트러블 값 찾기
            selected_reviews = df[df['상품명'] == selected_title]
            age = selected_reviews['나이'].mode().values[0]
            types = selected_reviews['피부타입'].mode().values[0]
            trouble = selected_reviews['피부트러블'].mode().values[0]

        print(f"{selected_title}은/는 {age}, {types} 타입이며, {trouble} 고민을 가진 분들이 많이 씁니다.")

        # 같은 값을 가지는 다른 상품 찾기
        similar_products = []

        for index, row in df.iterrows():
          product = row['상품명']
          if product != selected_title:  # 선택한 상품과 다른 상품만 처리
            other_age_counts = Counter(df[df['상품명'] == product]['나이'])
            other_types_counts = Counter(df[df['상품명'] == product]['피부타입'])
            other_trouble_counts = Counter(df[df['상품명'] == product]['피부트러블'])
            if (other_age_counts.most_common(1)[0][0] == age and
                other_types_counts.most_common(1)[0][0] == types and
                other_trouble_counts.most_common(1)[0][0] == trouble):
                similar_products.append(product)
    if similar_products:
        print("같은 값을 가지는 다른 상품:")
        for product in similar_products:
            print(product)
    else:
        print("같은 값을 가지는 다른 상품이 없습니다.")

In [ ]:
from datetime import datetime, timedelta

def total_reviews(df):
  while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        # 여러 키워드에 대한 처리
        keywords = title.split()
        condition = lambda x: all(keyword.lower() in x.lower() for keyword in keywords)
        # 중복 제거를 위해 unique() 사용
        matching_titles = [t for t in df['상품명'].unique().tolist() if condition(t)]
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        # 사용자가 상품을 선택하도록 함
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        # 선택한 상품의 리뷰 가져오기
        reviews = df[df['상품명'] == selected_title]
        age = reviews['나이'].mode().values[0]
        types = reviews['피부타입'].mode().values[0]
        trouble = reviews['피부트러블'].mode().values[0]

        reviews['리뷰작성날짜'] = pd.to_datetime(reviews['리뷰작성날짜'], format='%Y-%m-%d', errors='coerce')
        current_date = datetime.now()

        print("이 제품은 "+age+", "+types+"이면서 "+trouble+" 고민을 가진 사람들이 많이 쓰는 제품입니다.")

        one_year_ago = pd.Timestamp.now() - pd.DateOffset(years=1)
        recent_reviews = reviews[reviews['리뷰작성날짜'] > one_year_ago]

        # 리뷰 작성 날짜별로 리뷰 수 계산
        review_trend = recent_reviews.groupby(recent_reviews['리뷰작성날짜'].dt.to_period("M")).size().reset_index(name='리뷰수')
        review_trend['리뷰작성날짜'] = review_trend['리뷰작성날짜'].dt.to_timestamp()

        # 그래프로 표현
        plt.figure(figsize=(10, 6))
        plt.plot(review_trend['리뷰작성날짜'], review_trend['리뷰수'], marker='o')
        plt.title(f'{selected_title} - 최근 1년 리뷰 동향')
        plt.xlabel('리뷰 작성 날짜')
        plt.ylabel('리뷰 수')
        plt.grid(True)
        plt.show(block=False)

In [10]:
chatbot_with_recommendation(skincare)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 화산송이
일치하는 상품명 목록:
1. 화산송이 모공 바하 클렌징 폼 150g
2. 수퍼 화산송이 모공 마스크
3. 화산송이 모공 바하 클렌징폼  250g
상품을 선택하세요 (번호 입력): 1
화산송이 모공 바하 클렌징 폼 150g의 전체 리뷰 요약: 여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 세안 해주면 좋은점이 뾰루지가 안나고 뽀드득하게 씻겨줘서 좋고 피지과도할때 딥 클렌징폼이라 그런지 피부에 자극이 없는건 아니지만 격일로 쓰기에 좋은거같아요.
비슷한 상품을 보여드릴까요? (Y/N): y
화산송이 모공 바하 클렌징 폼 150g은/는 40대, 복합성 타입이며, 트러블 고민을 가진 분들이 많이 씁니다.


KeyboardInterrupt: ignored